In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from pandas.tseries.offsets import DateOffset

In [2]:
 data_file=pd.read_csv("C:/Users/adit/Desktop/wudi_project_2/assignment_data.csv")

In [3]:
item_codes=[3938,3960]

In [4]:
def prediction_df(data_file,item_codes):
    data_file['date']= pd.to_datetime(data_file['date'],dayfirst=True)
    filtered_data = data_file[~(data_file['billed_quantity'] < 0)]
    for i in range(len(item_codes)):
        item_id=filtered_data[filtered_data.item_id==item_codes[i]]
        item_id['tot_sales']=(item_id['billed_quantity']* item_id['sell_rate'])
        item_id=item_id.sort_values(by='date')
        item_id.reset_index(inplace = True, drop = True) 
        item_id['date'] = item_id['date'].dt.strftime('%Y-%m')
        new_df=item_id[['date','tot_sales']]
        df_zero=new_df.groupby('date',as_index=False).sum()
        
        
        df_zero['date']= pd.to_datetime(df_zero['date'])
        df_zero.columns=["Month","Sales"]
        df_zero.set_index('Month',inplace=True)
        df_zero['Sales First Difference'] = df_zero['Sales'] - df_zero['Sales'].shift(1)
        
        model=sm.tsa.statespace.SARIMAX(df_zero['Sales'],order=(1, 0, 1),seasonal_order=(1,1,0,12))
        results=model.fit()
        df_zero['forecast']=results.predict(start=22,end=29,dynamic=True)
        
        future_dates=[df_zero.index[-1]+ DateOffset(months=x)for x in range(0,10)]
        future_datest_df=pd.DataFrame(index=future_dates[1:],columns=df_zero.columns)
        
        future_df=pd.concat([df_zero,future_datest_df])
        future_df['forecast'] = results.predict(start = 25, end = 40, dynamic= True)  
        
        future_df.to_csv('C:/Users/adit/Desktop/wudi_project_2/prediction_df_item_id'+str(item_codes[i])+'.csv',index=False)

        
    return future_df
        

In [ ]:
prediction_df(data_file,item_codes)